# 6.数据聚合 (Groupby & Pivot)                             


#### 笨办法学 Python 数据分析  / learn data analysis the hard way
- @Author：知行并重
- @公众号：知行并重
- @Github 代码：https://github.com/kevin-meng/learn-data-analysis-the-hard-way


## 目录
1. **数据聚合 groupby**
    - 单函数 \ 多函数 \自定义函数
    - 聚合数据变形（拉长 or 变宽）

2. **数据透视表 pivot_table**
    - 使用 pivot_table
    - 对多重索引数据进行切片

3.  **数据透视表 ： crosstab (交叉表)**
    - 统计频数、频率
    - 按行行列统计频数
  
4.  多重索引数据的**重命名**

### Step1: 导入必要的库

In [4]:
import pandas as pd #数据分析
import numpy as np #科学计算

pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 一、数据读取
data = pd.read_csv("../input/titanic.csv")

data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


## 一、数据聚合

#### 聚合函数
* count()
* mean()
* std()
* agg()  ==>一次运用多个函数

### 1.1 查看数据中不同性别的生存情况

按照`Sex` 和 `Survived	`做汇总

In [8]:
# 两个变量关系
data_group1 = data.groupby(['Sex','Survived']).agg(len)
data_group1.head()

PassengerId  Pclass  Name    Age  SibSp  Parch  Ticket  \
Sex    Survived                                                           
female 0                  81      81    81  81.00     81     81      81   
       1                 233     233   233 233.00    233    233     233   
male   0                 468     468   468 468.00    468    468     468   
       1                 109     109   109 109.00    109    109     109   

                  Fare  Cabin  Embarked  
Sex    Survived                          
female 0         81.00     81        81  
       1        233.00    233       233  
male   0        468.00    468       468  
       1        109.00    109       109

In [6]:
# agg 默认会对所用的列都应用该函数，如果你想减少不必要的等待时间，可以提前选中需要的列
# data_group = data.groupby(['Sex','Survived'])[['PassengerId']].agg(len)

In [49]:
# 采用 size() count() 也可以打到同样的效果, 掌握上面常用的方法，可以记忆使用。
# data_group1 = data.groupby(['Sex','Survived']).size()

# 转换成数据框 dataframe
# data_group1.to_frame('num')

# data_group1 = data.groupby(['Sex','Survived'])[['Survived']].count()

### 1.2 查看数据中不同仓位等级中幸存与否的乘客的人数，平均票价，最高票价

统计'mean','count','sum'(groupby)。也可以是自定义的函数
- **多变量聚合、多函数同时运用**

In [9]:
def distinct(ls):
    return len(set(ls))

In [10]:
data_group2 = data[['Pclass','Survived','Fare']].groupby(['Pclass','Survived']).agg(['count','mean','max',distinct])
data_group2

Fare                      
                count  mean    max distinct
Pclass Survived                            
1      0           80 64.68 263.00    57.00
       1          136 95.61 512.33    68.00
2      0           97 19.41  73.50    27.00
       1           87 22.06  65.00    31.00
3      0          372 13.67  69.55    98.00
       1          119 13.69  56.50    55.00

### 1.3 数据拉长变宽
unstack()  or stack() 方法

有没有发现什么规律？

In [17]:
data_group3 = data_group2.unstack()
data_group3
# data_group3.stack('Survived')

Fare                                              
         count       mean          max        distinct      
Survived     0    1     0     1      0      1        0     1
Pclass                                                      
1           80  136 64.68 95.61 263.00 512.33    57.00 68.00
2           97   87 19.41 22.06  73.50  65.00    27.00 31.00
3          372  119 13.67 13.69  69.55  56.50    98.00 55.00

##  二、数据透视表 ： pivot_table  (数据透视图)

用过 Excel 的朋友应该不陌生的

### 2.1 查看数据中不同性别等级中幸存与否的乘客的人数

对某一变量求和,以`Sex`为行 `Survived`为列，以`PassengerId` 计数 为值

In [29]:
pivot1 = pd.pivot_table(data,index=['Sex'],columns=['Survived'],values=['PassengerId'], aggfunc=len )
pivot1

PassengerId     
Survived           0    1
Sex                      
female            81  233
male             468  109

如果加上 `margins =True` 参数后，对比结果有什么不同？

### 2.2 查看数据中不同仓位等级中幸存与否的乘客的人数，平均票价，最高票价

- **思考题**

对比`data_group2` 和 `pivot2` 程序有什么不同，结果是否有什么差异？

In [124]:
pivot2 = pd.pivot_table(data,
                        index=['Pclass'], \
                        columns=['Survived'], \
                        values=['Fare'], \
                        aggfunc=[len,np.mean,np.max] )
pivot2

len              mean               amax          
           Fare              Fare               Fare          
Survived      0      1          0          1       0         1
Pclass                                                        
1          80.0  136.0  64.684008  95.608029  263.00  512.3292
2          97.0   87.0  19.412328  22.055700   73.50   65.0000
3         372.0  119.0  13.669364  13.694887   69.55   56.4958

### 2.3 对透视表进行切片
- 可以首先进行切片 然后再进行 透视表
- 也可以先透视，再切片（筛选过滤） 相当Excel中的第四块内容 （本节重点）

#### 2.3.1 对数据表pivot1中的人数总和大于400的性别（列求和加总过滤）

In [64]:
pivot1[pivot1.sum(axis = 1) >400]
# 均值、最大、最小 等等都是可以
# pivot1[pivot1.mean(axis = 1) >200]
# pivot1[pivot1.min(axis = 1) >100]
# pivot1[pivot1.max(axis = 1) >400]


PassengerId     
Survived           0    1
Sex                      
male             468  109

#### 2.3.2 对于多重索引 (MultiIndex) 进行切片



In [83]:
pivot3 = pd.pivot_table(data_train,index=['Pclass','Sex'], \
                    columns=['Embarked','Survived'], \
                    values=['Fare'], \
                    aggfunc=[len,np.sum,np.mean] )
pivot3

len                                       sum             \
               Fare                                      Fare              
Embarked          C           Q            S                C              
Survived          0     1     0     1      0     1          0          1   
Pclass Sex                                                                 
1      female   1.0  42.0   NaN   1.0    2.0  46.0    28.7125  4943.8208   
       male    25.0  17.0   1.0   NaN   51.0  28.0  2021.1790  1907.3627   
2      female   NaN   7.0   NaN   2.0    6.0  61.0        NaN   176.8792   
       male     8.0   2.0   1.0   NaN   82.0  15.0   203.3458    50.8667   
3      female   8.0  15.0   9.0  24.0   55.0  33.0   117.2250   220.7583   
       male    33.0  10.0  36.0   3.0  231.0  34.0   287.7878   114.3584   

                                                              mean  \
                                                              Fare   
Embarked              Q                    S                     C   
Survived              0         1          0          1          0   
Pclass Sex                                                           
1      female       NaN   90.0000   303.1000  4450.1917  28.712500   
       male     90.0000       NaN  2731.7291  1451.3167  80.847160   
2      female       NaN   24.7000   109.5000  1358.6500        NaN   
       male     12.3500       NaN  1557.8000   307.7500  25.418225   
3      female   98.1417  242.0168  1208.2960   434.6708  14.653125   
       male    426.2958   38.7500  2947.2572   579.1373   8.720842   

                                                                        
                                                                        
Embarked                           Q                      S             
Survived                1          0          1           0          1  
Pclass Sex                                                              
1      female  117.710019        NaN  90.000000  151.550000  96.743298  
       male    112.197806  90.000000        NaN   53.563316  51.832739  
2      female   25.268457        NaN  12.350000   18.250000  22.272951  
       male     25.433350  12.350000        NaN   18.997561  20.516667  
3      female   14.717220  10.904633  10.084033   21.969018  13.171842  
       male     11.435840  11.841550  12.916667   12.758689  17.033450

#### 2.3.2.1 对行索引 `index.get_level_values`
选取女性的数据 

In [72]:
pivot3.iloc[(pivot3.index.get_level_values('Sex') == 'female'),:]  

# 选取 2\3等仓的数据
# pivot3.iloc[pivot3.index.get_level_values('Pclass').isin([2, 3]),:]  

len                                    sum                      \
              Fare                                   Fare                       
Embarked         C          Q           S               C                   Q   
Survived         0     1    0     1     0     1         0          1        0   
Pclass Sex                                                                      
1      female  1.0  42.0  NaN   1.0   2.0  46.0   28.7125  4943.8208      NaN   
2      female  NaN   7.0  NaN   2.0   6.0  61.0       NaN   176.8792      NaN   
3      female  8.0  15.0  9.0  24.0  55.0  33.0  117.2250   220.7583  98.1417   

                                                   mean              \
                                                   Fare               
Embarked                        S                     C               
Survived              1         0          1          0           1   
Pclass Sex                                                            
1      female   90.0000   303.100  4450.1917  28.712500  117.710019   
2      female   24.7000   109.500  1358.6500        NaN   25.268457   
3      female  242.0168  1208.296   434.6708  14.653125   14.717220   

                                                            
                                                            
Embarked               Q                      S             
Survived               0          1           0          1  
Pclass Sex                                                  
1      female        NaN  90.000000  151.550000  96.743298  
2      female        NaN  12.350000   18.250000  22.272951  
3      female  10.904633  10.084033   21.969018  13.171842

#### 2.3.2.2 对列索引 `columns.get_level_values`
选取在 C 港口上传的数据

In [74]:
pivot3.iloc[:,pivot2.columns.get_level_values('Embarked')=='C']

len              sum                  mean            
               Fare             Fare                  Fare            
Embarked          C                C                     C            
Survived          0     1          0          1          0           1
Pclass Sex                                                            
1      female   1.0  42.0    28.7125  4943.8208  28.712500  117.710019
       male    25.0  17.0  2021.1790  1907.3627  80.847160  112.197806
2      female   NaN   7.0        NaN   176.8792        NaN   25.268457
       male     8.0   2.0   203.3458    50.8667  25.418225   25.433350
3      female   8.0  15.0   117.2250   220.7583  14.653125   14.717220
       male    33.0  10.0   287.7878   114.3584   8.720842   11.435840

##### 2.3.2.3  其他方法

还可以借助 `query` 方法对透视表进行切片。我使用较少，供了解。

In [80]:
pivot3.query('Sex == "female"')

len                                    sum                      \
              Fare                                   Fare                       
Embarked         C          Q           S               C                   Q   
Survived         0     1    0     1     0     1         0          1        0   
Pclass Sex                                                                      
1      female  1.0  42.0  NaN   1.0   2.0  46.0   28.7125  4943.8208      NaN   
2      female  NaN   7.0  NaN   2.0   6.0  61.0       NaN   176.8792      NaN   
3      female  8.0  15.0  9.0  24.0  55.0  33.0  117.2250   220.7583  98.1417   

                                                   mean              \
                                                   Fare               
Embarked                        S                     C               
Survived              1         0          1          0           1   
Pclass Sex                                                            
1      female   90.0000   303.100  4450.1917  28.712500  117.710019   
2      female   24.7000   109.500  1358.6500        NaN   25.268457   
3      female  242.0168  1208.296   434.6708  14.653125   14.717220   

                                                            
                                                            
Embarked               Q                      S             
Survived               0          1           0          1  
Pclass Sex                                                  
1      female        NaN  90.000000  151.550000  96.743298  
2      female        NaN  12.350000   18.250000  22.272951  
3      female  10.904633  10.084033   21.969018  13.171842

##  三、数据透视表 ： crosstab  (交叉表)

交叉表是一种用于统计分组频率的特殊透视表。默认统计频数。

### 3.1 统计频数

#### 统计 Survived 与 Sex 之间频数分布关系

仅有两列时 怎么看 数量 ，和 每个的百分比. 频数 crosstab() 

In [81]:
pd.crosstab(data['Survived'],data['Sex'],margins =True)

Sex,female,male,All
Survived,,,
0,81,468,549
1,233,109,342
All,314,577,891


### 3.2  统计频率

对于频率会稍有不同，它会存在三种情况：

1. 按整体
2. 按行
3. 按列

#### 3.2.1 按整体频率  `normalize=True`

In [92]:
pd.crosstab(data['Survived'],data['Sex'],margins =True,normalize="all")

Sex,female,male,All
Survived,,,
0,0.090909,0.525253,0.616162
1,0.261504,0.122334,0.383838
All,0.352413,0.647587,1.000000


In [101]:
# 借助 apply 也可以实现对应效果
# pd.crosstab(data['Survived'], data['Sex']).apply(lambda r: r/len(data))

#### 3.2.2 按行计算频率 

In [93]:
pd.crosstab(data['Survived'],data['Sex'],margins =True,normalize="index")

Sex,female,male
Survived,,
0,0.147541,0.852459
1,0.681287,0.318713
All,0.352413,0.647587


In [96]:
# 借助 apply 也可以实现对应效果
# pd.crosstab(data['Survived'],data['Sex']).apply(lambda r: r/r.sum(), axis=1)  

#### 3.2.3 按列计算频率 

In [95]:
pd.crosstab(data['Survived'],data['Sex'],margins =True,normalize="columns")

Sex,female,male,All
Survived,,,
0,0.257962,0.811092,0.616162
1,0.742038,0.188908,0.383838


In [98]:
# 借助 apply 也可以实现对应效果
# pd.crosstab(data['Survived'],data['Sex']).apply(lambda r: r/r.sum(), axis=0)  

#### 3.2.4 了解函数的其他参数

pd.crosstab??

### 四、多重索引的重命名（补充）

对于像 聚合函数 `groupby` 透视表 `pivot_table` 以及 交叉表 `crosstab`得到的数据通常列名都是多重索引。
后续使用选取的时候会不太方便，这时需要对其进行重命名。比如像上面的 `pivot2` 数据举例


#### 4.1 查看索引

In [125]:
pivot2.columns
# 三重索引

MultiIndex([( 'len', 'Fare', 0),
            ( 'len', 'Fare', 1),
            ('mean', 'Fare', 0),
            ('mean', 'Fare', 1),
            ('amax', 'Fare', 0),
            ('amax', 'Fare', 1)],
           names=[None, None, 'Survived'])

In [126]:
pivot2

len              mean               amax          
           Fare              Fare               Fare          
Survived      0      1          0          1       0         1
Pclass                                                        
1          80.0  136.0  64.684008  95.608029  263.00  512.3292
2          97.0   87.0  19.412328  22.055700   73.50   65.0000
3         372.0  119.0  13.669364  13.694887   69.55   56.4958

#### 4.2 重命名

In [127]:
cols = [f"{x[0]}_{x[1]}_{x[2]}" for x in pivot2.columns]  # 如果你知道有几重索引的话
cols

['len_Fare_0',
 'len_Fare_1',
 'mean_Fare_0',
 'mean_Fare_1',
 'amax_Fare_0',
 'amax_Fare_1']

In [128]:
# 更一般的做法： 
# 当索引中含有 数值型的值时 需转化为字符串
# ["_".join([str(i) for i in x]) for x in pivot2.columns] 

In [129]:
# 重命名列名
pivot2.columns = cols

In [130]:
pivot2

,len_Fare_0,len_Fare_1,mean_Fare_0,mean_Fare_1,amax_Fare_0,amax_Fare_1
Pclass,,,,,,
1,80.0,136.0,64.684008,95.608029,263.00,512.3292
2,97.0,87.0,19.412328,22.055700,73.50,65.0000
3,372.0,119.0,13.669364,13.694887,69.55,56.4958


# 谢谢观看
![](../pics/thankyou.png)
